In [1]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
import numpy as np
# from osgeo import gdal_array
# from osgeo import gdalnumeric

gdal.UseExceptions()

In [2]:
enc = ogr.Open('data/ENC_ROOT/US4MA13M/US4MA13M.000')
assert enc.GetDriver().GetDescription() == 'S57' # S57
m_covr = enc.GetLayerByName('M_COVR')

In [3]:
bathy = gdal.Open('data/navd_bath_30m')
assert bathy.GetDriver().GetDescription() == 'AIG' # ArcInfo Binary Grid
assert bathy.RasterCount == 1
bathy_band = bathy.GetRasterBand(1)
bathy_np = bathy_band.ReadAsArray()
bathy_np[bathy_np == bathy_band.GetNoDataValue()] = np.NaN

In [4]:
bathy_np.shape

(4885, 3551)

In [5]:
np.count_nonzero(np.isnan(bathy_np))

8705556

In [6]:
for f in m_covr:
    if f.GetField('CATCOV') != 22:
        g = f.GetGeometryRef()
        print(g.GetEnvelope())
        #print(g.ExportToWkt())

(-71.0826389, -70.0329167, 42.0333333, 42.6336111)
(-71.1146088, -69.9994574, 42.0009322, 42.6657174)


In [8]:
geotransform = bathy.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Origin = (330893.590626, 4789330.06)
Pixel Size = (30.0, -30.0)


In [ ]:
line = bathy_band.ReadRaster(xoff=0, yoff=0, xsize=100, ysize=1)
line